In [33]:
import scipy.io as scio
import numpy as np    
import matplotlib.pyplot as plt
import sys
import os
import math
from scipy.misc import imsave

In [34]:
path='Dataset/Subject_new_'
labeled_images = []
labeled_colorimages = []
images_modified=[]
min_image = [190, 187, 165, 141, 122, 117, 123, 147, 191, 191, 194, 65, 70, 80, 72, 66, 66, 70, 75, 72, 65, 76, 114, 121, 123, 119, 123, 121, 116, 114, 123, 127, 124, 100, 89, 94, 97, 88, 87, 97, 116, 122, 120, 118, 116, 105, 90, 70, 53, 46, 50, 72, 107, 121, 115, 110, 108, 117, 126, 120, 115, 106, 110, 121, 122, 112, 88, 91, 110, 105, 102, 87, 79, 92, 116, 115, 107, 134, 126, 117, 114, 116, 118, 120, 120, 133, 128, 129, 122, 120, 119, 112, 105, 94, 94, 92, 92, 83, 76, 161, 162, 149, 135, 130, 125, 120, 123, 137, 149, 154]
max_image = [294, 298, 303, 307, 306, 306, 303, 300, 301, 298, 300, 192, 194, 195, 198, 199, 197, 198, 197, 197, 198, 195, 224, 227, 227, 228, 226, 232, 235, 239, 239, 238, 242, 240, 243, 246, 249, 247, 245, 244, 242, 239, 236, 235, 228, 232, 239, 241, 240, 242, 241, 240, 237, 232, 229, 243, 244, 249, 252, 253, 257, 256, 256, 253, 251, 249, 205, 206, 204, 206, 212, 212, 211, 207, 209, 213, 214, 229, 231, 235, 236, 237, 239, 241, 243, 240, 234, 233, 208, 214, 214, 218, 220, 222, 224, 227, 230, 228, 231, 284, 285, 285, 287, 286, 286, 285, 287, 285, 284, 283]
diff_image = [104, 111, 138, 166, 184, 189, 180, 153, 110, 107, 106, 127, 124, 115, 126, 133, 131, 128, 122, 125, 133, 119, 110, 106, 104, 109, 103, 111, 119, 125, 116, 111, 118, 140, 154, 152, 152, 159, 158, 147, 126, 117, 116, 117, 112, 127, 149, 171, 187, 196, 191, 168, 130, 111, 114, 133, 136, 132, 126, 133, 142, 150, 146, 132, 129, 137, 117, 115, 94, 101, 110, 125, 132, 115, 93, 98, 107, 95, 105, 118, 122, 121, 121, 121, 123, 107, 106, 104, 86, 94, 95, 106, 115, 128, 130, 135, 138, 145, 155, 123, 123, 136, 152, 156, 161, 165, 164, 148, 135, 129]
max_rows = 216
resized_images = []
resized_colorimages = []
resized_labeledimages = []
weighted_images = []

In [35]:
def load_subject(subject_no):
    local_path=path+subject_no+'/Sub_'+subject_no+'.mat'
    sub = scio.loadmat(local_path)
    print (sub.keys())
    return sub
    
def reshape(ndarray):
    ndarray_reshaped = np.array(ndarray.tolist())
    ndarray_reshaped = ndarray_reshaped.reshape(ndarray.shape).transpose(2,0,1)
    print (ndarray_reshaped.shape)
    return ndarray_reshaped

def get_gt_ids(subject_no):
    local_path=path+subject_no+'/gt_id.mat'
    gt_id = scio.loadmat(local_path)
    print (gt_id.keys())
    gt_id_list = gt_id['fr1']
    print(gt_id_list)
    gt_list = []
    for item in gt_id_list:
        gt_list.append(item[0])
    return gt_list
    
def crop_image(image,left_bound,right_bound):
    image = image[:,left_bound:right_bound]
    return image

def resize_image(image,min_value,max_value,diff_value):
    extra = max_rows - diff_value
    min_value = int(min_value - math.ceil(extra/2.0))
    max_value = int(max_value + math.floor(extra/2.0))
    image = image[min_value:max_value,:]
    return image

In [36]:
for sub_index in range(10):
    sub=load_subject(str(sub_index+1))
    sub_images = sub['images']
    images_reshaped = reshape(sub_images)
    manualLayer = sub['manualLayers1']
    manualLayer_reshaped=reshape(manualLayer)
    left_bounds = [136,121,117,109,137,146,134,120,124,140]
    right_bounds = [636,621,617,609,637,646,634,620,624,640]
    gt_id_list = []
    gt_id_list = get_gt_ids(str(sub_index+1))
    for i in gt_id_list:
        #min_value = sys.maxint 
        #max_value = -sys.maxint -1
        test = manualLayer_reshaped[i-1]
        test=crop_image(test,left_bounds[sub_index],right_bounds[sub_index])
        image_modified=crop_image(images_reshaped[i-1],left_bounds[sub_index],right_bounds[sub_index])
        images_modified.append(image_modified)
        for j in range(8):
            for k in range(500):
                if(test[j][k]==0):
                    test[j][k] = test[j][k-1]
        res_image = np.zeros((496,500))
        color = np.zeros((496,500,3))
        for j in range(7):
            for k in range(500):
                for a in range(test[j][k], test[j+1][k]):
                        res_image[a][k] = j+1
        labeled_images.append(res_image)
        for j in range(496):
            for k in range(500):
                if(res_image[j][k]==0):
                    color[j][k] = [0,0,0]
                if(res_image[j][k]==1):
                    color[j][k] = [128,0,0]
                if(res_image[j][k]==2):
                    color[j][k] = [0,128,0]
                if(res_image[j][k]==3):
                    color[j][k] = [128,128,0] 
                if(res_image[j][k]==4):
                    color[j][k] = [0,128,128]
                if(res_image[j][k]==5):
                    color[j][k] = [64,0,0]
                if(res_image[j][k]==6):
                    color[j][k] = [192,0,0]
                if(res_image[j][k]==7):
                    color[j][k] = [128,64,64]
                if(res_image[j][k]==9):
                    color[j][k] = [0,64,128]
        labeled_colorimages.append(color)
        
        '''for k in range(500):
            min_value = min(test[0][k],min_value)
            max_value = max(test[7][k],max_value)
        diff_value = max_value - min_value
        min_image.append(min_value)
        max_image.append(max_value)
        diff_image.append(diff_value)'''
        
            

['automaticLayersNormal', 'automaticLayersDME', 'grader2', 'manualLayers1', '__header__', '__globals__', 'manualLayers2', 'grader1', 'images', 'grader_SJ', '__version__']
(61, 496, 768)
(61, 8, 768)
['fr1', '__version__', '__header__', '__globals__']
[[11]
 [16]
 [21]
 [26]
 [29]
 [31]
 [33]
 [36]
 [41]
 [46]
 [51]]
['automaticLayersNormal', 'automaticLayersDME', 'grader2', 'manualLayers1', '__header__', '__globals__', 'manualLayers2', 'grader1', 'images', 'grader_SJ', '__version__']
(61, 496, 768)
(61, 8, 768)
['fr1', '__version__', '__header__', '__globals__']
[[11]
 [16]
 [21]
 [26]
 [29]
 [31]
 [33]
 [36]
 [41]
 [46]
 [51]]
['automaticLayersNormal', 'automaticLayersDME', 'grader2', 'manualLayers1', '__header__', '__globals__', 'manualLayers2', 'grader1', 'images', 'grader_SJ', '__version__']
(61, 496, 768)
(61, 8, 768)
['fr1', '__version__', '__header__', '__globals__']
[[11]
 [16]
 [21]
 [26]
 [29]
 [31]
 [33]
 [36]
 [41]
 [46]
 [51]]
['automaticLayersNormal', 'automaticLayersDME'

In [37]:
for i in range(110):
    image = images_modified[i]
    labeled_image = labeled_images[i]
    labeled_colorimage = labeled_colorimages[i]
    resized_image = resize_image(image,min_image[i],max_image[i],diff_image[i])
    resized_labeled_image = resize_image(labeled_image,min_image[i],max_image[i],diff_image[i])
    resized_colorimage = resize_image(labeled_colorimage,min_image[i],max_image[i],diff_image[i])
    resized_images.append(resized_image)
    resized_labeledimages.append(resized_labeled_image)
    resized_colorimages.append(resized_colorimage)

In [38]:
imgplot = plt.imshow(resized_colorimages[50])
plt.show()


In [40]:
print(resized_images[50])

[[35 42 49 ...,  0  0  0]
 [31 39 43 ..., 16 14 10]
 [41 47 45 ..., 21 19 13]
 ..., 
 [61 59 58 ..., 53 55 54]
 [62 56 57 ..., 57 56 53]
 [62 56 57 ..., 57 55 49]]


In [41]:
for item in range(110):
    imsave('Dataset/Resized/resized_'+str(item+1)+'.png',resized_images[item])
for item in range(110):
    imsave('Dataset/Resized_Colored/resized_colored'+str(item+1)+'.png',resized_colorimages[item])

In [10]:
np.save('resized_images.npy',resized_images)
np.save('resized_labeledimages.npy',resized_labeledimages)

In [50]:
count = [0]*8
weighted_images = []
for i in range(110):
    image = resized_labeledimages[i]
    weighted_image = np.zeros((216,500))
    for j in range(216):
        for k in range(500):
             if(image[j][k]==1):
                w2 = 11.459
             elif(image[j][k] == 2):
                w2 = 5.63
             elif(image[j][k]== 3):
                w2 = 11.007 
             elif(image[j][k] == 4):
                w2 = 14.368 
             elif(image[j][k]== 5):
                w2 = 3.336 
             elif(image[j][k]== 6):
                w2 = 13.647 
             elif(image[j][k]== 7):
                w2 = 16.978 
             else:
                w2 = 0
             if(j!=0 and j!=215):
                if(image[j+1][k]-image[j-1][k]>0 and w2<>0):
                    w1 = 15 
                   # count = count +1
                    count[int(image[j-1][k])] = count[int(image[j-1][k])] + 1 
                else:
                    w1 = 0
             else:
                w1 = 0
             w = 1 + w1 + w2
             weighted_image[j][k] = w
    weighted_images.append(weighted_image)

In [51]:
for i in range(7):
    print(count[i])

55000
109637
109701
109553
108494
109937
109758


In [52]:
for item in range(110):
    imsave('Dataset/Weighted/weighted_'+str(item+1)+'.png',weighted_images[item])

In [53]:
np.save('weighted_images.npy',np.array(weighted_images))

In [54]:
imgplot = plt.imshow(weighted_images[2])
plt.show()

In [55]:
print(len(resized_images))
print(resized_images[0].shape)

110
(216, 500)


In [56]:
resized_cropped_images = []
resized_cropped_labeledimages = []
weighted_cropped_images = []
resized_color_train_images = []

In [57]:
for i in range(110):
    for j in range(7):
        temp = crop_image(resized_images[i],j*64,(j+1)*64)
        resized_cropped_images.append(temp)
        imsave('Dataset/Resized_train/train_'+str((i*7)+(j+1))+'.png',temp)

In [58]:
for i in range(110):
    for j in range(7):
        temp = crop_image(resized_labeledimages[i],j*64,(j+1)*64)
        resized_cropped_labeledimages.append(temp)
        imsave('Dataset/Resized_labeledtrain/resized_labeledimage_'+str((i*7)+(j+1))+'.png',temp)

In [59]:
for i in range(110):
    for j in range(7):
        temp = crop_image(weighted_images[i],j*64,(j+1)*64)
        weighted_cropped_images.append(temp)
        imsave('Dataset/Weighted_cropped/weighted_image_'+str((i*7)+(j+1))+'.png',temp)

In [60]:
for i in range(110):
    for j in range(7):
        temp = crop_image(resized_colorimages[i],j*64,(j+1)*64)
        resized_color_train_images.append(temp)
        imsave('Dataset/Resized_color_train/resized_color_train_image_'+str((i*7)+(j+1))+'.png',temp)

In [61]:
np.save('resized_cropped_images.npy',np.array(resized_cropped_images))
np.save('resized_cropped_labeledimages.npy',np.array(resized_cropped_labeledimages))
np.save('weighted_cropped_images.npy',np.array(weighted_cropped_images))
